# Sample Julia Notebook - downloading and using data from S3
[![Open In Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/aws/studio-lab-examples/blob/main/use-julia-in-studio-lab/2-sample-julia-nb.ipynb)

## Installing packages

As we did in the Installing Julia notebook, we use the Pkg.add command to add a few packages that we will use for this notebook.

 - [Gadfly](http://gadflyjl.org/stable/): grammar of graphics plotting, similar to ggplot in R
 - [AWS](https://github.com/JuliaCloud/AWS.jl): third-party SDK for performing tasks in AWS
 - [AWSS3](https://github.com/JuliaCloud/AWSS3.jl): high-level third-party interface to Amazon Simple Storage Service
 - [DataFrames](https://dataframes.juliadata.org/stable/): data structure for working with tabular data, similar to Pandas in Python
 - [Query](http://www.queryverse.org/Query.jl/stable/): convenience functions for working with DataFrames and other queryable data sources, similar to dplyr in R.
 - [JSON](https://github.com/JuliaIO/JSON.jl): library for parsing JSON data
 - [TranscodingStreams](https://juliaio.github.io/TranscodingStreams.jl/latest/): framework for working with encoded data
 - [CodecZlib](https://github.com/JuliaIO/CodecZlib.jl): zlib compression codec for use with TranscodingStreams; allows us to work with gzipped data.
 
After installing the packages, we'll load them as we use them in this notebook

In [ ]:
using Pkg
Pkg.add(["Gadfly", "AWS", "AWSS3", "DataFrames", "Query", "JSON", "TranscodingStreams", "CodecZlib"])

## Example plots using Gadfly

First up, we'll plot a histogram of some normally-distributed random numbers. Then, we'll plot an analogous 2 dimensional density plot of some normally-distributed random numbers.

In [ ]:
using Gadfly

# Note that Gadfly is JIT compiled, which means that it can take a little time
# for the first plot to render. Subsequent plots are speedy.
plot(x = randn(10000),
     Geom.histogram)

In [ ]:
plot(x = randn(1000),
     y = randn(1000),
     Geom.density2d)

## Accessing and using data from Amazon S3

Here we will use the third-party AWS SDK and high-level S3 interface. The data we will use [are listed at the Registry of Open Data on AWS (RODA)](https://registry.opendata.aws/lei/), and are stored in a public S3 bucket s3://gleif/ in the eu-central-1 region. This dataset uniquely identifies legal entities using a code, and also provides information about ownership structures.

We will access these data anonymously, which can be done by creating and using an `aws_config` structure with `creds=nothing`. Note that we must also supply the bucket's region with our configuration.

Our first operation will be to list out all the keys in this S3 bucket that begin with "data/json/lei". We can see that the data are organized by date and time and have a ".json.gz" extension, denoting that these data are json formatted and compressed using gzip.

In [ ]:
using AWS
using AWSS3 # high-level interface to S3

# This AWSConfig instance allows us to make anonymous / unsigned requests
aws = global_aws_config(;creds = nothing, region = "eu-central-1")

# s3_list_keys returns a generator that we can collect immediately using collect
obj = collect(s3_list_keys(aws, "gleif", "data/json/lei/"; delimiter = ""))

Next, we'll load one of the compressed JSON objects from S3. Since it is gzipped, we'll need to decompress the object. And because the object is also in JSON format, we'll need to parse the JSON to work with it in Julia.

Objects in S3 can be very large ([up to 5TB in a single object as of this writing](https://docs.aws.amazon.com/AmazonS3/latest/userguide/qfacts.html)). Processing larger objects may require an incremental strategy, however in this case the objects we're interested in can safely fit in memory. We therefore download, decompress, and parse the whole object at once.

In [ ]:
using JSON, TranscodingStreams, CodecZlib

rr = JSON.parse(String(transcode(GzipDecompressor, read(S3Path("s3://gleif/data/json/rr/date=2021-12-10/time=00:00/20211210-0000-gleif-goldencopy-rr.json.gz")))))

The resulting dictionary object contains a "relations" key that contains a number of records pertaining to the relationships between legal entities. To help us understand the structure of an individual record, we can print the first record as JSON.

In [ ]:
print(json(rr["relations"][1], 4)) # the 4 tells, the json library to indent using 4 spaces

To help us do some simple analysis of these relationship records, we'll construct a DataFrame containing just a handful fields that we're interested in. Here we construct the DataFrame from named columns of vectors extracted from the dictionary we generated from the JSON from S3.

In [ ]:
using DataFrames, Query

relations = DataFrame(
    startnode = map(x -> x["RelationshipRecord"]["Relationship"]["StartNode"]["NodeID"]["\$"], rr["relations"]),
    endnode = map(x -> x["RelationshipRecord"]["Relationship"]["EndNode"]["NodeID"]["\$"], rr["relations"]),
    relationshiptype = map(x -> x["RelationshipRecord"]["Relationship"]["RelationshipType"]["\$"], rr["relations"])
)

Once data are loaded into a DataFrame object, it is straightforward to perform various table operations. Here we use macros from the Query package to refine our DataFrame as a pipeline. Here we limit the results to rows where the relationshiptype is "IS_DIRECTLY_CONSOLIDATED_BY", then group by values in the endnode column, then get a count of records per group, and lastly sort the table in descending order.

In [ ]:
group_sizes = relations |>
    @filter(_.relationshiptype == "IS_DIRECTLY_CONSOLIDATED_BY") |>
    @groupby(_.endnode) |>
    @map({endnode=key(_), count = length(_)}) |>
    @orderby_descending(_.count) |>
    DataFrame

If we wanted to join names for all these groups, we could extract that information from the LEI data similarly housed on S3. Or if we are just casually interested in the top item, we can also [search for a LEI record on GLEIF's website](https://search.gleif.org/#/search/simpleSearch=784F5XWPLTWKTBV3E584&fulltextFilterId=LEIREC_FULLTEXT&currentPage=1&perPage=15&expertMode=false#search-form). In this case, the largest direct consolidator is The Goldman Sachs Group, Inc.

Lastly, we can plot the distribution of group sizes, once again using the Gadfly plotting library. Here we see that most direct consolidators have far fewer entries—one or two—and that there is a very long tail of larger direct consolidators.

In [ ]:
plot(group_sizes, x=:count, Geom.density)